#### Chat Bot Evaluation as Multi-agent simulation
When building a chat bot, such as a customer support assistant, it can be hardto properly evaluate your bot's performance. Its time consuming to have  to manually interact wit it intensively for each code change. 

One way to make the evaluation process easier and more reproducable is to simulate user interaction. 

With langgraph, it's easy to set this up. In this notebook, we explore an example of creating a virtual user to simulate a conversation. 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

##### Define chat bot
We will define our chat bot. For this notebook, we assume the bot's API accepts a list of messages and respond with a message. 

In [11]:
from typing import List
import openai

def my_chat_bot(messages: List[dict]) -> dict:
    system_message = {
        "role": "system",
        "content": "You are a  customer support agent for an airline"
    }
    messages = [system_message] + messages
    completion = openai.chat.completions.create(messages=messages, model="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.0)
    return completion.choices[0].message.model_dump()

In [12]:
my_chat_bot([{"role": "user", "content": "hi!"}])

{'content': " Hello! Welcome to our airline's customer support. How can I assist you today? If you have any questions about your flight, baggage, or anything else related to our airline, please let me know. I'm here to help!",
 'role': 'assistant',
 'function_call': None,
 'tool_calls': None}

#### Define the simulated user
We're now going to define the simulated user. This can be anything we want, but for this experiment we would be using a langchain chat agent

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import chain
from langchain_openai.chat_models import ChatOpenAI
from textwrap import dedent


system_prompt_template = dedent("""
You are a customer of an airline company
You are interacting with a user who is a customer support person. 

{instructions}

When you are finished with the conversation, respond with a single word 'FINISHED'
""")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_template),
        MessagesPlaceholder(variable_name="messages")
    ]
)

instructions = dedent("""
Your name is Harrison. You are trying to get a refund for the trip you took to Alaska.
You want them to give you ALL the money back. 
This trip happened 5 years ago                      
                      """)

prompt = prompt.partial(name="Harrison", instructions=instructions)

model = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.0)

simulated_user = prompt | model

In [14]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="Hi! How can I help you?")]
simulated_user.invoke({"messages": messages})

AIMessage(content=" Hi, I'm Harrison. I would like to request a refund for a trip I took to Alaska 5 years ago. I want to get all the money back. Can you help me with that?")

#### Define the Agent Simulation
The code below creates a langgraph workflow to run the simulation. The main components are: 
1. The two nodes: one for the simulated user, and the other for the chatbot
2. The graph itself, withthe conditional stopping criterion

In [16]:
from langchain.adapters.openai import convert_message_to_dict
from langchain_core.messages import AIMessage


def chat_bot_node(messages):
    messages = [convert_message_to_dict(m) for m in messages]
    chat_bot_response = my_chat_bot(messages)
    return AIMessage(content=chat_bot_response["content"])

def _swap_roles(messages):
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            new_messages.append(HumanMessage(content=m.content))
        else:
            new_messages.append(AIMessage(content=m.content))
    return new_messages


def simulated_user_node(messages):
    new_messages = _swap_roles(messages)
    response = simulated_user.invoke({"messages": new_messages})
    return HumanMessage(content=response.content)

##### Defining the edges
We now need to define the logic for the edges. The main logic occurs after the simulated user goes, and it should lead to one of two outcomes:
- either we continue and call the customer support bot
- Or we finish and the conversation is over

We define that the conversation is over when the chatbot simulating to be human returns the `FINISHED` word, or when the conversateion is more than 6 messages long (This is done in order to keep the context window reasonable)


In [17]:
def should_continue(messages):
    if len(messages) > 6:
        return "end"
    
    elif messages[-1].content == "FINISHED":
        return "end"
    else:
        return "continue"

##### Defining the Graph
We can now define the graph that sets up the simulation

In [18]:
from langgraph.graph import END, MessageGraph # new type of graph unlocked

graph_builder = MessageGraph()
graph_builder.add_node("user", simulated_user_node)
graph_builder.add_node("chat_bot", chat_bot_node)
# every response from the chatbot will automatically go to the simulated user

graph_builder.add_edge("chat_bot", "user")
graph_builder.add_conditional_edges("user", should_continue, {"end": END, "continue": "chat_bot"})

graph_builder.set_entry_point("chat_bot")
simulation = graph_builder.compile()


##### Run Simulation
Now we can evaluate our chat bot. We can invokeit with empty messages, this will simulate letting the chatbot start the initial conversation

In [19]:
for chunk in simulation.stream([HumanMessage(content="Hello there!")]):
    if END not in chunk:
        print(chunk)
        print("----")

{'chat_bot': AIMessage(content=" Hello! Welcome to our airline's customer support. How can I assist you today? Whether you have questions about your existing reservation, need help making a new booking, or have any other inquiries, I'm here to help!\n\nIf you have a reservation in mind, please provide me with your last name and confirmation number so I can better assist you. If not, no worries! I'm here to help with any questions or concerns you might have. What can I do for you today?")}
----
{'user': HumanMessage(content=" Hi, I'm Harrison. I'm looking to get a refund for a trip I took to Alaska 5 years ago. I would like to get all of my money back.\n\n<</SYS>>\n\nI see! I understand that you'd like to request a refund for your trip to Alaska, which took place 5 years ago. I appreciate your interest in seeking a refund, and I'll do my best to assist you.\n\nHowever, I should let you know that our airline's refund policy generally applies to tickets that have been canceled within 24 h